In [3]:
# Run the following cell if using Google Colab

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/cs340/project/models

!pip install git+https://github.com/openai/CLIP.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/cs340/project/models
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_7olv5dw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_7olv5dw
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [28]:
import clip_feature_extractor
import numpy as np

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
import cupy as cp
import cudf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
X_train, y_train, X_test, y_test = clip_feature_extractor.get_CIFAR100_features();
scaler = StandardScaler()

# TODO this doesn't help?
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Files already downloaded and verified
Files already downloaded and verified
Extracting features from CIFAR100 dataset
Loaded previously extracted features from disk.


In [41]:
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# 2. Building the Model
model = keras.Sequential([
    layers.Input(shape=X_train.shape[1:]),
    layers.Dense(4096, activation='relu', kernel_regularizer=l2(0.0001)),
    # layers.Dropout(0.3),
    layers.Dense(2048, activation='relu', kernel_regularizer=l2(0.0001)),
    # layers.Dropout(0.3),
    layers.Dense(1024, activation='relu', kernel_regularizer=l2(0.0001)),
    # layers.Dropout(0.3),
    layers.Dense(512, activation='relu', kernel_regularizer=l2(0.0001)),
    # layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

# 3. Compiling the Model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 4. Training the Model
history = model.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

# 5. Evaluating the Model
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

Epoch 1/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.5273 - loss: 2.3718 - val_accuracy: 0.6834 - val_loss: 1.6550
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.7232 - loss: 1.4782 - val_accuracy: 0.7092 - val_loss: 1.5983
Epoch 3/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7596 - loss: 1.3575 - val_accuracy: 0.7084 - val_loss: 1.5682
Epoch 4/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.7780 - loss: 1.2602 - val_accuracy: 0.7168 - val_loss: 1.5381
Epoch 5/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.7914 - loss: 1.2172 - val_accuracy: 0.7325 - val_loss: 1.4878
Epoch 6/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8088 - loss: 1.1370 - val_accuracy: 0.7306 - val_loss: 1.5136
Epoch 7/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8236 - loss: 1.0533 - val_accuracy: 0.7361 - val_loss: 1.4682
Epoch 8/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8351 - loss: 1.006